In [1]:
import sys
sys.path.append("../")

In [8]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0
def apply_signal(row):
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE        

In [6]:
    pair = "EUR_USD"
    df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)

In [9]:
df_an = MACD(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [10]:
df_an.head(1)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100
0,2016-01-13 03:00:00+00:00,450,1.0829,1.08324,1.08247,1.0825,1.08282,1.08316,1.0824,1.08243,1.08297,1.08331,1.08254,1.08258,-0.001062,-0.000931,-0.000131,1.086813


In [11]:
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0:
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0:
        return SELL
    return NONE

In [12]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross,axis=1)

In [13]:
df_an.direction.value_counts()

direction
 0    34175
 1     1473
-1     1472
Name: count, dtype: int64

In [14]:
def run_pair(pair,time_d=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = MACD(df_an)
    
    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()

    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross,axis=1)
        
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
           'direction', 'EMA_100' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d=time_d
    )
    
    gt.run_test()
    return gt.df_results
    

In [15]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p,time_d=1)))

prepare_data...
run_test...
Result: -554.5
prepare_data...
run_test...
Result: -53.0
prepare_data...
run_test...
Result: 22.0
prepare_data...
run_test...
Result: -124.5


In [16]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -554.5
EUR_USD -53.0
GBP_JPY 22.0
USD_CAD -124.5


In [17]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p,time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())

prepare_data...
run_test...
Result: -56.0
prepare_data...
run_test...
Result: 12.0
prepare_data...
run_test...
Result: 26.5
prepare_data...
run_test...
Result: -39.0
AUD_NZD -56.0
EUR_USD 12.0
GBP_JPY 26.5
USD_CAD -39.0
